In [1]:
# ***** NOTA BENE! *****
# perché %matplotlib widget funzioni, installare nell'ambiente virtuale 
# il pacchetto ipympl con il comando:
# pip install ipympl
#
# ATTENZIONE: perché funzioni è necessario chiudere e rilanciare jupyter-lab
#
# STILE DI VISUALIZZAZIONE PLOT FATTI CON MATPLOTLIB
%matplotlib widget
#
#
import pandas as pd
import numpy as np
from sklearn import datasets
from sklearn.decomposition import PCA
import matplotlib
import matplotlib.pyplot as plt

# Importazione del Dataset “Labeled Faces in the Wild”

Il dataset in questione è disponibile tramite il sottomodulo "dataset" di scikit-learn (come il dataset iris, wine e molti altri) e la funzione fetch_lfw_people. Documentazione completa della funzione qui: https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_lfw_people.html#sklearn.datasets.fetch_lfw_people

Il dataset caricato dalla funzione restituisce (ormai vecchie) immagini di volti famosi.
L'oggetto restituito è della classe "Bunch" di scikit-learn, dal funzionamento simile ai dizionari (per come lo utilizzeremo, potrebbe essere di fatto un dizionario).

In [2]:
# Il codice presente in questa cella serve nel caso si verifichi un errore del tipo
#
# "URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1124)>"
#
# al momento di chiamare la funzione fetch_lfw_people di sklearn.datasets
#
# ATTENZIONE: il codice di questa cella non è quindi sempre necessario; se quindi non lo fosse, commentarlo pure.
#

import os, ssl

if (not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None)):
    ssl._create_default_https_context = ssl._create_unverified_context

## Importazione di parte del dataset e prime ispezioni del contenuto

Di seguito importiamo parte del dataset, stampiamo la descrizione ufficiale fornita da scikit-learn ed esploriamo un po' il contenuto.

### Importazione e Stampa Descrizione

In [3]:
lfw_people = datasets.fetch_lfw_people(min_faces_per_person=70, resize=0.4)

print(lfw_people['DESCR'])

.. _labeled_faces_in_the_wild_dataset:

The Labeled Faces in the Wild face recognition dataset
------------------------------------------------------

This dataset is a collection of JPEG pictures of famous people collected
over the internet, all details are available on the official website:

    http://vis-www.cs.umass.edu/lfw/

Each picture is centered on a single face. The typical task is called
Face Verification: given a pair of two pictures, a binary classifier
must predict whether the two images are from the same person.

An alternative task, Face Recognition or Face Identification is:
given the picture of the face of an unknown person, identify the name
of the person by referring to a gallery of previously seen pictures of
identified persons.

Both Face Verification and Face Recognition are tasks that are typically
performed on the output of a model trained to perform Face Detection. The
most popular model for Face Detection is called Viola-Jones and is
implemented in the OpenC

### Visualizzazioni "Esplorative"

Prima di tutto, per comodità, salviamo in variabili apposite i dati contenuti nel simil-dizionario.

In [4]:
face_data = lfw_people['data']
face_images = lfw_people['images']
face_tnames = lfw_people['target_names']
face_targets = lfw_people['target']


Per praticità, creiamoci dei DataFrame di lavoro

**ESERCIZIO:** Completare il codice della cella seguente per creare il DataFrame "face_targets_df" avente per colonne "target", con i valori di "face_targets", e "target_names", avente come valori le stringhe di nomi in "face_tnames" corrispondenti ai valori nella colonna accanto.

In [5]:
# DataFrame delle immagini rappresentate come vettori
face_data_df = pd.DataFrame(face_data)

# DataFrame delle classi di appartenenza e dei nomi
face_targets_df = pd.DataFrame({'target': face_targets, 'target_names':[face_tnames[t] for t in face_targets]})

In [6]:
face_data_df.head()

,0,1,2,3,4,5,6,7,8,9,...,1840,1841,1842,1843,1844,1845,1846,1847,1848,1849
0,254.333328,254.000000,252.000000,240.333328,185.333328,143.666672,174.000000,197.000000,195.666672,192.000000,...,102.000000,100.000000,99.000000,96.000000,92.333336,90.333336,89.333336,87.000000,88.666664,87.000000
1,39.000000,50.666668,47.000000,54.333332,98.666664,121.000000,139.666672,157.666672,171.000000,177.333328,...,44.333332,59.666668,73.333336,77.333336,83.666664,103.666664,122.666664,117.666664,115.000000,133.000000
2,89.666664,103.666664,126.333336,141.333328,151.666672,155.333328,155.333328,159.666672,163.000000,166.666672,...,125.333336,112.000000,100.000000,106.666664,125.333336,117.666664,125.000000,175.333328,183.333328,182.666672
3,16.666666,8.000000,6.666667,6.000000,16.333334,70.000000,170.333328,170.000000,161.000000,106.333336,...,126.000000,190.000000,230.000000,238.000000,239.333328,233.666672,134.000000,28.000000,38.333332,51.333332
4,123.000000,121.000000,127.000000,129.333328,129.333328,134.666672,142.000000,143.000000,147.666672,152.333328,...,31.333334,17.333334,6.333333,5.333333,5.666667,10.333333,10.666667,22.000000,61.333332,70.333336


In [7]:
face_targets_df.head()

,target,target_names
0,5,Hugo Chavez
1,6,Tony Blair
2,3,George W Bush
3,1,Colin Powell
4,0,Ariel Sharon


 
 
 Vediamo la distribuzione dei volti all'interno del dataset.
 
 **ESERCIZIO:** Completare il codice nella cella seguente per visualizzare la distribuzione dei volti nel dataset.
 
 

In [8]:
# Definire face_counts sfruttando il metodo "value_counts" delle Serie rispetto alla 
# colonna 'target_names' di face_targets_df
face_counts =face_targets_df.value_counts(subset='target_names')

plt.figure()
plt.bar(np.arange(len(face_counts.values)),face_counts.values)
plt.xticks(ticks=np.arange(len(face_counts.values)), 
          labels=face_counts.index.to_list(),
          rotation=15)
plt.title('DISTRIBUZIONE VOLTI')
plt.grid()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Visualizziamo alcuni dei volti, sia come immagine che come vettore.

In [9]:
# Raccogliamo gli indici corrispondenti alle varie classi in un dizionario

class_indices = {}

for name in face_tnames:
    class_indices[name] = face_targets_df.loc[face_targets_df['target_names']==name].index.to_list()

In [10]:
# Visualizziamo il primo dato per ogni classe

fig1, axs1 = plt.subplots(len(face_tnames), 2, figsize=(8, len(face_tnames) * 3))
                         #numero righe,  colonne, dimensione immagine
for i in range(len(face_tnames)):
    
    name = face_tnames[i]
    first_ind = class_indices[name][0]
    
    axs1[i, 0].bar(np.arange(face_data.shape[1]), face_data[first_ind, :])
    #asse 0 dove inserire titolo e grafico
    axs1[i, 0].set_title('{} - vettore R^{}'.format(name, face_data.shape[1]))
    
    axs1[i, 1].imshow(face_images[first_ind], cmap=plt.cm.gray)
    #mostra l'immagine                        ,scala di grigi
    axs1[i, 1].set_title('{} - immagine/matrice R^{}x{}'.format(name, face_images.shape[1], face_images.shape[2]))

plt.subplots_adjust(wspace=0.5, hspace=0.5)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## PCA ed Eigenfaces

La PCA è stata applicata (soprattutto in passato, prima dell'avvento del Deep Learning) a problemi di riconoscimento facciale. Tutt'oggi può essere utile per questi scopi o, per esempio, per la compressione di immagini.

Proviamo quindi ad applicare la PCA ai vettori contenuti nella variabile "face_data".

### Indagine Preliminare sulla Varianza Spiegata

**ATTENZIONE:** in questo caso i vettori hanno come elementi i pixel delle immagini e assumono tutti valori tra 0 (nero) e 255 (bianco). Non si rende necessaria l'applicazione di uno StandardScaler (o simili).

**ESERCIZIO:** Completare il codice della cella seguente per visualizzare l'andamento della varianza spiegata (percentuale) all'aumentare delle PC considerate.


In [11]:
# Inizializzazione oggetto PCA
pca_global = PCA()
# "Fit" dell'oggetto PCA
pca_global.fit(face_data)

# Definizione di un array della varianza spiegata percentuale CUMULATIVA
expld_variance_global = np.insert(np.cumsum(pca_global.explained_variance_ratio_),0,0)

plt.figure()
plt.plot(expld_variance_global)
plt.title('LFW')
plt.xticks(ticks=np.arange(1, pca_global.n_features_ + 1, 100),
           labels=[f'PC{i}' for i in range(1, pca_global.n_features_ + 1, 100)],
           rotation=30)
plt.xlabel('Principal components')
plt.ylabel('Cumulative explained variance')
plt.grid()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Guardando la curva della varianza spiegata, osserviamo che con un numero abbastanza ridotto di PC possiamo "spiegare" una buona percentuale della varianza totale.

Prendiamo come esempio per l'esercizio una percentuale di varianza totale spiegata pari all'95%.

**ESERCIZIO:** Completare il codice della cella seguente per implementare una PCA che selezioni un numero di PC corrispondenti al 95% di varianza totale spiegata.

In [12]:
pca_percentage = 0.95
pca = PCA(pca_percentage)
pca.fit(face_data)

print('Numero PC: {}'.format(pca.n_components_))
print('% Varianza Tot. Spiegata: {}'.format(pca.explained_variance_ratio_.sum()))

Numero PC: 142
% Varianza Tot. Spiegata: 0.950280487537384


### Eigenfaces

Le cosiddette "eigenfaces" altro non sono che le PC (autovettori della matrice di covarianza del dataset) visualizzate come immagini invece che come vettori.

In [13]:
# Salviamo le PC "risistemate" come matrici/immagini in una lista apposita.

_, height, width = face_images.shape
eigenfaces = [pca.components_[i].reshape((height, width)) for i in range(pca.n_components_)]

vmin = pca.components_.min()
vmax = pca.components_.max()

vmin = min([vmin, -vmax, 0.0])
vmax = max([-vmin, vmax, 0.0])

print('vmin: {} (dark purple)'.format(vmin))
print('vmax: {} (yellow)'.format(vmax))

# Numero di Eigenfaces da visualizzare (non tutte per allegerire l'esecuzione dei plot)
n_pc_tosee = 5

for i in range(n_pc_tosee): 
    fig, axs = plt.subplots(1, 3, figsize=(10, 4))
    axs[0].bar(np.arange(pca.components_.shape[1]), pca.components_[i, :])
    axs[0].set_title('PC{} - vettore R^{}'.format(i + 1, pca.components_.shape[1]))
    
    axs[1].imshow(eigenfaces[i], cmap=plt.cm.gnuplot2, vmin=vmin, vmax=vmax)
    axs[1].set_title('PC{} - eigenface (vmin/vmax)'.format(i + 1, height, width))
    
    axs[2].imshow(eigenfaces[i], cmap=plt.cm.gray)
    axs[2].set_title('PC{} - eigenface (normalized)'.format(i + 1, height, width))
    
    fig.suptitle('PC{} - {}% Var. Spiegata'.format(i + 1, np.around(100 * pca.explained_variance_ratio_[i], decimals=2)))
    
    plt.show()


vmin: -0.14722168445587158 (dark purple)
vmax: 0.14722168445587158 (yellow)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Ricostruzione volti da Eigenfaces

Vediamo come i volti presenti nel dataset vengono ricostruiti usando il numero di eigenfaces (cioè PC) che abbiamo selezionato (se le usassimo tutte e 1850, avremmo una ricostruzione esatta).

**ESERCIZIO:** Completare il codice seguente per la comparare le immagini originali e la loro ricostruzione fatta usando il numero di PC scelto.

In [14]:
# Selezione di "n_randsamples" volti random dal dataset

n_randsamples = 25
i_rand = np.random.choice(face_data.shape[0], n_randsamples, replace=False)

# Definire una matrice di n_randsamples volti scelti secondo gli indici i_rand (una riga, un volto)
rand_faces = face_data[i_rand,:]

# Definire la matrice "built_randfaces" rappresentante la matrice rand_faces ricostruita tramite le PC.
# SUGGERIMENTO: usare i metodi "transform" e "inverse_transform" della PCA. 
built_randfaces = pca.inverse_transform(pca.transform(rand_faces))

print('Sample Indices: {}'.format(i_rand))

for i in range(n_randsamples):
    fig, axs = plt.subplots(1, 2, figsize=(8, 3))
    ii = i_rand[i]
    face_ii = face_images[ii]
    builtface_ii = built_randfaces[i, :]
    builtface_ii = builtface_ii.reshape((height, width))
    
    axs[0].imshow(face_ii, cmap=plt.cm.gray)
    axs[0].set_title('Volto {} ({})'.format(ii, face_targets_df.loc[ii, 'target_names']))
    
    axs[1].imshow(builtface_ii, cmap=plt.cm.gray)
    axs[1].set_title('Volto {} ricostruito'.format(ii))
    
plt.show()

Sample Indices: [ 928  397  401  818 1026   34  995  301   49 1089  961  502 1256  581
  808 1198  646  172 1240  718 1002  935  711   32   99]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …